# Putative Human – Mouse BRCA1 Orthologs  
Write a program using NCBI's E-Utilities to retrieve the ids of RefSeq human BRCA1 proteins from NCBI. 
* Use the query: "Homo sapiens"[Organism] AND BRCA1[Gene Name] AND REFSEQ
* Extend your program to search these protein ids (one at a time) vs RefSeq proteins (refseq_protein) using the NCBI blast web-service.
* Further extend your program to filter the results for significance (E-value < 1.0e-5) and to extract mouse sequences (match "Mus musculus" in the description).Note: you may need to 
* Request at least 200 alignments from qblast to see the first mouse protein (keyword parameter hitlist_size, default is 50), or __Restrict the qblast search to mouse refseq proteins (keyword parameter entrez_query)__

Need to use the [Biopython guide](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc91)

Questions
* extend to refseq: search both gb and refseq? or search genes then proteins all in refseq?

In [6]:
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
import os.path

Entrez.email = 'michael.chambers2@nih.gov'

In [ ]:
handle = Entrez.esearch(
    db='protein',
    term = '"Homo sapiens"[Organism] AND BRCA1[Gene Name] AND REFSEQ',
    usehistory='y'    
)

result = Entrez.read(handle)
handle.close()

id_list = ','.join(result['IdList'])
handle = Entrez.efetch(
    db='protein',
    id=id_list,
    rettype='gb'
)

for gi,r in zip(result['IdList'], SeqIO.parse(handle, 'genbank')):
    print(f'\n*** START: {gi} ***\n')
    print('GI:', gi)
    print('Accession:', r.id)
    print('Description:', r.description)

    print(f"\nBLAST for GI {gi}...\n")
    result_handle = NCBIWWW.qblast(
        'blastp',
        'refseq_protein',
        gi,
        expect=1e-5,
        entrez_query='"Mus musculus"[Organism]'
    )
    
    blast_results = result_handle.read()
    result_handle.close()
    
    # parse the file (was 38)
    for blast_result in NCBIXML.parse(blast_results):
        for desc in blast_results.descriptions:
            print('***Alignment***')
            print('Sequence:',desc.title)
            print('Evalue:', desc.e)
            print()
    
    file = f'blastp-np-{gi}.xml'    
    save_file = open(file, 'w')
    save_file.write(blast_results)
    
    print(f'\n*** END {gi} ***')


*** START: 6552299 ***

GI: 6552299
Accession: NP_009225.1
Description: breast cancer type 1 susceptibility protein isoform 1 [Homo sapiens]


In [15]:
# use Esearch history
handle = Entrez.esearch(
    db='protein',
    term = '"Homo sapiens"[Organism] AND BRCA1[Gene Name] AND REFSEQ',
    usehistory='y'
)
result = Entrez.read(handle)
handle.close()

id_list = ','.join(result['IdList'])
handle = Entrez.efetch(
    db='protein',
    id=id_list,
    rettype='gb'
)
for gi,r in zip(result['IdList'], SeqIO.parse(handle, 'genbank')):
    print('GI:', gi)
    print('Accession:', r.id)
    print('Description', r.description)
    print()

# now blast for the results and save to file

    file = f'blastp-np-{gi}.xml'

    result_handle = NCBIWWW.qblast(
        'blastp',
        'refseq_protein',
        gi,
        expect=1e-5, # filter 
        entrez_query='"Mus musculus"[Organism]'
    )
    blast_results = result_handle.read()
    result_handle.close()

    save_file = open(file, 'w')
    save_file.write(blast_results)

GI: 6552299
Accession: NP_009225.1
Description breast cancer type 1 susceptibility protein isoform 1 [Homo sapiens]
GI: 237681125
Accession: NP_009230.2
Description breast cancer type 1 susceptibility protein isoform 5 [Homo sapiens]
GI: 237681123
Accession: NP_009229.2
Description breast cancer type 1 susceptibility protein isoform 4 [Homo sapiens]
GI: 237681121
Accession: NP_009228.2
Description breast cancer type 1 susceptibility protein isoform 3 [Homo sapiens]
GI: 237681119
Accession: NP_009231.2
Description breast cancer type 1 susceptibility protein isoform 2 [Homo sapiens]


In [ ]:
# need to get a genbank record

In [1]:
import os.path
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIWWW
from Bio.GenBank.Record import Record

Entrez.email = 'michael.chambers2@nih.gov'

In [23]:
handle = Entrez.esearch(db="protein", term='"Homo sapiens"[Orgn] AND BRCA1[Gene Name] AND REFSEQ', usehistory="y")

result = Entrez.read(handle)
handle.close()
count = int(result["Count"])
session_cookie = result["WebEnv"]
query_key = result["QueryKey"]

#print(count, session_cookie, query_key)

# Get the results in chunks of 100
chunk_size = 100

query_dict = {}

#print("Search Protein DB")
for chunk_start in range(0,count,chunk_size) :
    handle = Entrez.efetch(db="protein", rettype="gb", retstart=chunk_start, retmax=chunk_size, webenv=session_cookie, query_key=query_key)
    for r in SeqIO.parse(handle,"genbank"):
        print(type(r))

        
        query_dict[r.id] = r.seq
    handle.close()

<class 'Bio.SeqRecord.SeqRecord'>
<class 'Bio.SeqRecord.SeqRecord'>
<class 'Bio.SeqRecord.SeqRecord'>
<class 'Bio.SeqRecord.SeqRecord'>
<class 'Bio.SeqRecord.SeqRecord'>


In [2]:
query_dict

{'NP_009225.1': Seq('MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQ...SHY', IUPACProtein()),
 'NP_009230.2': Seq('MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQ...ADV', IUPACProtein()),
 'NP_009229.2': Seq('MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQ...SHY', IUPACProtein()),
 'NP_009228.2': Seq('MLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEY...SHY', IUPACProtein()),
 'NP_009231.2': Seq('MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQ...SHY', IUPACProtein())}

In [29]:
# search blast
for key,val in query_dict.items():
    prefix, suffix = key.split('_')
    print(prefix.lower())
    print(suffix)
    file_name = f'blastp-{prefix.lower()}-{suffix}.xml'
    #if not os.path.exists(file_name):
    result_handle = NCBIWWW.qblast("blastp", prefix.lower(), val)
    blast_results = result_handle.read()
    result_handle.close()

    save_file = open(file_name, 'w')
    save_file.write(blast_results)
    save_file.close()
    

np
009225.1


AttributeError: '_io.StringIO' object has no attribute 'colse'

In [ ]:
#for key,val in query_dict.items():
key = "NP_009225.1"
val = "MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQKTSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEFCNKSKQPGLARSQHNRWAGSKETCNDRRTPSTEKKVDLNADPLCERKEWNKQKLPCSENPRDTEDVPWITLNSSIQKVNEWFSRSDELLGSDDSHDGESESNAKVADVLDVLNEVDEYSGSSEKIDLLASDPHEALICKSERVHSKSVESNIEDKIFGKTYRKKASLPNLSHVTENLIIGAFVTEPQIIQERPLTNKLKRKRRPTSGLHPEDFIKKADLAVQKTPEMINQGTNQTEQNGQVMNITNSGHENKTKGDSIQNEKNPNPIESLEKESAFKTKAEPISSSISNMELELNIHNSKAPKKNRLRRKSSTRHIHALELVVSRNLSPPNCTELQIDSCSSSEEIKKKKYNQMPVRHSRNLQLMEGKEPATGAKKSNKPNEQTSKRHDSDTFPELKLTNAPGSFTKCSNTSELKEFVNPSLPREEKEEKLETVKVSNNAEDPKDLMLSGERVLQTERSVESSSISLVPGTDYGTQESISLLEVSTLGKAKTEPNKCVSQCAAFENPKGLIHGCSKDNRNDTEGFKYPLGHEVNHSRETSIEMEESELDAQYLQNTFKVSKRQSFAPFSNPGNAEEECATFSAHSGSLKKQSPKVTFECEQKEENQGKNESNIKPVQTVNITAGFPVVGQKDKPVDNAKCSIKGGSRFCLSSQFRGNETGLITPNKHGLLQNPYRIPPLFPIKSFVKTKCKKNLLEENFEEHSMSPEREMGNENIPSTVSTISRNNIRENVFKEASSSNINEVGSSTNEVGSSINEIGSSDENIQAELGRNRGPKLNAMLRLGVLQPEVYKQSLPGSNCKHPEIKKQEYEEVVQTVNTDFSPYLISDNLEQPMGSSHASQVCSETPDDLLDDGEIKEDTSFAENDIKESSAVFSKSVQKGELSRSPSPFTHTHLAQGYRRGAKKLESSEENLSSEDEELPCFQHLLFGKVNNIPSQSTRHSTVATECLSKNTEENLLSLKNSLNDCSNQVILAKASQEHHLSEETKCSASLFSSQCSELEDLTANTNTQDPFLIGSSKQMRHQSESQGVGLSDKELVSDDEERGTGLEENNQEEQSMDSNLGEAASGCESETSVSEDCSGLSSQSDILTTQQRDTMQHNLIKLQQEMAELEAVLEQHGSQPSNSYPSIISDSSALEDLRNPEQSTSEKAVLTSQKSSEYPISQNPEGLSADKFEVSADSSTSKNKEPGVERSSPSKCPSLDDRWYMHSCSGSLQNRNYPSQEELIKVVDVEEQQLEESGPHDLTETSYLPRQDLEGTPYLESGISLFSDDPESDPSEDRAPESARVGNIPSSTSALKVPQLKVAESAQSPAAAHTTDTAGYNAMEESVSREKPELTASTERVNKRMSMVVSGLTPEEFMLVYKFARKHHITLTNLITEETTHVVMKTDAEFVCERTLKYFLGIAGGKWVVSYFWVTQSIKERKMLNEHDFEVRGDVVNGRNHQGPKRARESQDRKIFRGLEICCYGPFTNMPTDQLEWMVQLCGASVVKELSSFTLGTGVHPIVVVQPDAWTEDNGFHAIGQMCEAPVVTREWVLDSVALYQCQELDTYLIPQIPHSHY"
    
prefix, suffix = key.split('_')
print(prefix.lower())
print(suffix)
file_name = f'blastp-{prefix.lower()}-{suffix}.xml'

result_handle = NCBIWWW.qblast(
    "blastp", 
    "np", 
    val, # this can also be a seq object
    entrez_query='"Mus musculus"[Organism]'
)
blast_results = result_handle.read()
result_handle.close()

save_file = open(file_name, 'w')
save_file.write(blast_results)
save_file.close()

np
009225.1


In [36]:
import os.path
from Bio.Blast import NCBIWWW

key = "NR_8332116"
prefix, suffix = key.split('_')
print(prefix.lower())
print(suffix)

file_name = f'blastp-{prefix.lower()}-{suffix}.xml'
print(file_name)
if not os.path.exists(file_name):
    result_handle = NCBIWWW.qblast("blastn", prefix.lower(), suffix)
    blast_results = result_handle.read()
    result_handle.close()

    save_file = open(file_name, 'w')
    save_file.write(blast_results)
    save_file.close()

nr
8332116
blastp-nr-8332116.xml


In [4]:
NCBIWWW.qblast?

Signature:
NCBIWWW.qblast(
    program,
    database,
    sequence,
    url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi',
    auto_format=None,
    composition_based_statistics=None,
    db_genetic_code=None,
    endpoints=None,
    entrez_query='(none)',
    expect=10.0,
    filter=None,
    gapcosts=None,
    genetic_code=None,
    hitlist_size=50,
    i_thresh=None,
    layout=None,
    lcase_mask=None,
    matrix_name=None,
    nucl_penalty=None,
    nucl_reward=None,
    other_advanced=None,
    perc_ident=None,
    phi_pattern=None,
    query_file=None,
    query_believe_defline=None,
    query_from=None,
    query_to=None,
    searchsp_eff=None,
    service=None,
    threshold=None,
    ungapped_alignment=None,
    word_size=None,
    short_query=None,
    alignments=500,
    alignment_view=None,
    descriptions=500,
    entrez_links_new_window=None,
    expect_low=None,
    expect_high=None,
    format_entrez_query=None,
    format_object=None,
    format_type='XML',
    nc

In [5]:
1e-5

1e-05